<a href="https://colab.research.google.com/github/sharon-nasimiyu/Automation-and-Prediction-of-data/blob/main/CPI_PREDICTION_AND_CATEGORIZATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Import required libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from math import sqrt
from google.colab import drive

In [11]:
drive.mount('/content/drive')
excel_path = '/content/drive/MyDrive/cpi_weights.xlsx'
csv_path = '/content/drive/MyDrive/CPI_Historic_Values_Zindi_Feb_23.csv'
cpi_weights = pd.read_excel(excel_path)
CPI_Historic = pd.read_csv(csv_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
CPI_Historic.head(2)

,Month,Category,Value,Percentage Change (From Prior Month),Unnamed: 4
0,28-02-2023,Headline_CPI,107.9,0.7,NaN
1,28-02-2023,Food and non-alcoholic beverages,115.6,1.0,NaN


In [13]:
cpi_weights.head(2)

,Category,Weight
0,Headline_CPI,0.00
1,Alcoholic beverages and tobacco,6.26


In [14]:
# list all the values in the 'Category' in cpi weights data set
name_values = cpi_weights['Category'].values
print(name_values)

['Headline_CPI' 'Alcoholic beverages and tobacco' 'Clothing and footwear'
 'Communication' 'Education' 'Food and non-alcoholic beverages' 'Health'
 'Household contents and services' 'Housing and utilities'
 'Miscellaneous goods and services' 'Recreation and culture'
 'Restaurants and hotels ' 'Transport']


In [15]:
# list all the values in the 'Category' in CPI_Historic data set
name_values1 = CPI_Historic['Category'].values
print(name_values)

['Headline_CPI' 'Alcoholic beverages and tobacco' 'Clothing and footwear'
 'Communication' 'Education' 'Food and non-alcoholic beverages' 'Health'
 'Household contents and services' 'Housing and utilities'
 'Miscellaneous goods and services' 'Recreation and culture'
 'Restaurants and hotels ' 'Transport']


In [16]:
#merging cpi weights to historic since to enhance historic since the category column is common in both
CPI_Enhanced = pd.merge( CPI_Historic, cpi_weights, on='Category', how='left')
CPI_Enhanced.head(2)

,Month,Category,Value,Percentage Change (From Prior Month),Unnamed: 4,Weight
0,28-02-2023,Headline_CPI,107.9,0.7,NaN,0.00
1,28-02-2023,Food and non-alcoholic beverages,115.6,1.0,NaN,17.14


In [18]:
# shape of the data
CPI_Enhanced.shape

(182, 6)

In [19]:
# size of the data
CPI_Enhanced.size

1092

In [22]:
#dealing with nans
#getting the number of nans
CPI_Enhanced.isna().sum()

Month                                     0
Category                                  0
Value                                     0
Percentage Change (From Prior Month)      0
Unnamed: 4                              182
Weight                                    0
dtype: int64

In [23]:
#since the whole unamed column doesnt have any data consider dropping
CPI_Enhanced = CPI_Enhanced.drop('Unnamed: 4', axis=1)
CPI_Enhanced.head(2)

,Month,Category,Value,Percentage Change (From Prior Month),Weight
0,28-02-2023,Headline_CPI,107.9,0.7,0.00
1,28-02-2023,Food and non-alcoholic beverages,115.6,1.0,17.14


In [26]:
#getting unique  values in each column
num_unique =CPI_Enhanced.nunique()
print(num_unique)

Month                                   14
Category                                13
Value                                   92
Percentage Change (From Prior Month)    37
Weight                                  13
dtype: int64


In [36]:
# convert the Month column to datetime format
CPI_Enhanced['Month'] = pd.to_datetime(CPI_Enhanced['Month'], format='%d-%m-%Y')

# group the CPI_Enhanced by month
grouped_monthly = CPI_Enhanced.groupby(['Month']).count().reset_index()
grouped_monthly

,Month,Category,Value,Percentage Change (From Prior Month),Weight
0,2022-01-31,13,13,13,13
1,2022-02-28,13,13,13,13
2,2022-03-31,13,13,13,13
3,2022-04-30,13,13,13,13
4,2022-05-31,13,13,13,13
5,2022-06-30,13,13,13,13
6,2022-07-31,13,13,13,13
7,2022-08-31,13,13,13,13
8,2022-09-30,13,13,13,13
9,2022-10-31,13,13,13,13


basically from the data it shows that everymonth had all the 13 categories

In [42]:
# get statistics of data
stats = CPI_Enhanced.describe()

print(stats)

            Value  Percentage Change (From Prior Month)      Weight
count  182.000000                            182.000000  182.000000
mean   104.013736                              0.474725    7.692308
std      3.480979                              0.901637    7.217995
min     98.900000                             -3.500000    0.000000
25%    101.200000                              0.000000    2.620000
50%    103.700000                              0.300000    4.370000
75%    105.275000                              0.700000   14.350000
max    115.600000                              4.800000   24.490000


In [61]:
# Convert non-numeric features to numeric using label encoding
from sklearn.preprocessing import LabelEncoder

# Select the non-numeric column to encode
non_numeric_col = 'Category'

# Create a label encoder object
label_encoder = LabelEncoder()

# Fit the encoder to the non-numeric column
label_encoder.fit(CPI_Enhanced[non_numeric_col])

# Transform the non-numeric column using the fitted encoder
CPI_Enhanced[non_numeric_col] = label_encoder.transform(CPI_Enhanced[non_numeric_col])


In [62]:
# Split data into training and testing sets
X = CPI_Enhanced.drop(['Value', 'Month'], axis=1)
y = CPI_Enhanced['Value']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [63]:
# Train linear regression model
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

In [64]:
# Train linear regression model
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

In [65]:
# Evaluate model on test set
y_pred = regressor.predict(X_test)
rmse = sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: ", rmse)



RMSE:  2.7283360033760555


In [67]:
# Predict next CPI value and sub-categories for 3 consecutive months
last_date = CPI_Enhanced['Month'].iloc[-1]
next_date = pd.date_range(start=last_date, periods=4, freq='MS')
next_dates = next_date.strftime("%Y-%m-%d").tolist()

next_cpi = regressor.predict(X.iloc[-3:])
next_subcategories = X.columns.tolist()



In [68]:
# Print predictions
print("Next CPI values for the next 3 months:")
for i in range(3):
    print(next_dates[i], next_cpi[i])

print("CPI sub-categories for the next 3 months:")
for i in range(3):
    print(next_dates[i], next_subcategories)


Next CPI values for the next 3 months:
2022-02-01 102.96156932187229
2022-03-01 104.73582343037965
2022-04-01 105.22291570674628
CPI sub-categories for the next 3 months:
2022-02-01 ['Category', 'Percentage Change (From Prior Month)', 'Weight']
2022-03-01 ['Category', 'Percentage Change (From Prior Month)', 'Weight']
2022-04-01 ['Category', 'Percentage Change (From Prior Month)', 'Weight']
